In [1]:
import numpy as np
import logging
import sys
from matplotlib import pyplot as plt

%matplotlib inline
sys.path.append("../")

import torch
from torchvision.datasets import MNIST
from torchvision import transforms

from aef.models.autoencoder import LinearAutoencoder, ConvolutionalAutoencoder, DenseAutoencoder
from aef.trainer import AutoencoderTrainer
from aef.losses import nll, mse

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)



## Data

In [2]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist = MNIST('./data', download=True, transform=img_transform)

## Train autoencoder

In [3]:
ae = ConvolutionalAutoencoder()

In [4]:
trainer = AutoencoderTrainer(ae, output_filename="../figures/training/conv/conv_phase1")
trainer.train(
    dataset=mnist,
    loss_functions=[mse],
    loss_labels=["MSE"],
    batch_size=256,
    epochs=10,
    verbose="all",
    initial_lr=1.e-3,
    final_lr=1.e-5
)

11:07 aef.trainer          INFO    Epoch   1: train loss  0.38395 (MSE:  0.384)
11:07 aef.trainer          INFO               val. loss   0.21578 (MSE:  0.216)
11:08 aef.trainer          INFO    Epoch   2: train loss  0.19542 (MSE:  0.195)
11:08 aef.trainer          INFO               val. loss   0.18595 (MSE:  0.186)
11:09 aef.trainer          INFO    Epoch   3: train loss  0.18027 (MSE:  0.180)
11:09 aef.trainer          INFO               val. loss   0.17450 (MSE:  0.174)
11:10 aef.trainer          INFO    Epoch   4: train loss  0.17107 (MSE:  0.171)
11:10 aef.trainer          INFO               val. loss   0.16751 (MSE:  0.168)
11:11 aef.trainer          INFO    Epoch   5: train loss  0.16563 (MSE:  0.166)
11:11 aef.trainer          INFO               val. loss   0.16341 (MSE:  0.163)
11:12 aef.trainer          INFO    Epoch   6: train loss  0.16247 (MSE:  0.162)
11:12 aef.trainer          INFO               val. loss   0.16106 (MSE:  0.161)
11:14 aef.trainer          INFO    Epoch

(array([0.38395358, 0.19541754, 0.1802651 , 0.17107154, 0.16563057,
        0.16247281, 0.16066718, 0.1595705 , 0.15894521, 0.15852129]),
 array([0.21577781, 0.18595452, 0.17449756, 0.16751294, 0.16340848,
        0.16105921, 0.15999088, 0.15908281, 0.15843756, 0.15820056]))

In [5]:
trainer = AutoencoderFlowTrainer(ae, output_filename="../figures/training/conv/conv_phase2")
trainer.train(
    dataset=mnist,
    loss_functions=[nll],
    loss_labels=["NLL"],
    batch_size=256,
    epochs=10,
    verbose="all",
    initial_lr=1.e-3,
    final_lr=1.e-5,
    parameters=ae.flow.parameters()
)

NameError: name 'AutoencoderFlowTrainer' is not defined

## Visualize latent space

In [ ]:
x = torch.cat([mnist[i][0].unsqueeze(0) for i in range(1000)], dim=0)
y = np.asarray([mnist[i][1] for i in range(1000)])

In [ ]:
h, u = ae.latent(x)
x_out = ae.decoder(h)
h = h.detach().numpy().reshape((-1,2))
u = u.detach().numpy().reshape((-1,2))

In [ ]:
fig = plt.figure(figsize=(10,5))

ax = plt.subplot(1,2,1)
for i in range(10):
    plt.scatter(h[y==i][:,0], h[y==i][:,1], c="C{}".format(i), s=20., label="{}".format(i+1))
plt.legend()
    
ax = plt.subplot(1,2,2)
for i in range(10):
    plt.scatter(u[y==i][:,0], u[y==i][:,1], c="C{}".format(i), s=20., label="{}".format(i+1))
plt.legend()

plt.tight_layout()
plt.show()


## Visualize reconstruction

In [ ]:
x = torch.cat([mnist[i][0].unsqueeze(0) for i in range(1000)], dim=0)
y = np.asarray([mnist[i][1] for i in range(1000)])

In [ ]:
h = ae.encoder(x)
x_out = ae.decoder(h)
h = h.detach().numpy()
x_out = x_out.detach().numpy()

In [ ]:
fig = plt.figure(figsize=(12,12))

for i in range(18):
    ax = plt.subplot(6, 6, 2*i + 1)
    plt.imshow(x[i].reshape((28,28)))
    ax = plt.subplot(6, 6, 2*i + 2)
    plt.imshow(x_out[i].reshape((28,28)))
        
plt.tight_layout()
plt.show()